# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
!pip install gmaps

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [30]:
load_csv = "Cities.csv"
city_df = pd.read_csv(load_csv)
city_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,temp,Wind Speed
0,victoria,100,HK,1639988494,88,22.2855,114.1577,17.40,0.45
1,mar del plata,0,AR,1639988679,96,-38.0023,-57.5575,17.71,3.13
2,upernavik,100,GL,1639988680,70,72.7868,-56.1549,-3.88,1.95
3,liyang,58,CN,1639988681,36,29.7057,118.2862,16.17,1.54
4,nikolskoye,100,RU,1639988682,85,59.7035,30.7861,-9.06,4.57


In [39]:
city_df.describe()

,Cloudiness,Date,Humidity,Lat,Lng,temp,Wind Speed
count,552.000000,5.520000e+02,552.000000,552.000000,552.000000,552.000000,552.000000
mean,59.050725,1.639989e+09,72.882246,19.532274,13.546494,10.636395,3.552156
std,40.165280,1.432679e+02,20.995780,33.908877,90.604217,17.137088,2.664705
min,0.000000,1.639988e+09,6.000000,-54.800000,-179.166700,-45.240000,0.000000
25%,15.000000,1.639989e+09,62.000000,-9.843675,-65.730675,0.530000,1.540000
50%,75.000000,1.639989e+09,77.000000,24.602200,16.011700,15.880000,2.855000
75%,99.000000,1.639989e+09,89.000000,46.666475,88.086775,24.200000,4.835000
max,100.000000,1.639989e+09,100.000000,78.218600,179.316700,36.910000,15.240000


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [38]:
gmaps.configure(api_key = g_key)

#get locations and humidity
city_locations = city_df[['Lat', 'Lng']].astype(float)
humidity = city_df['Humidity'].astype(float)

#format layout of map
layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'}

#add heatmap
fig = gmaps.figure(map_type='HYBRID', layout=layout)

heat_layer = gmaps.heatmap_layer(city_locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [44]:
myFavoriteWeather = city_df.loc[(city_df['temp'] >= 25) & (city_df['temp'] <= 35) & 
                                  (city_df['Wind Speed'] <= 20) & (city_df['Humidity'] <= 30)]
myFavoriteWeather

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,temp,Wind Speed
92,malakal,97,SS,1639988753,20,9.5334,31.6605,25.46,4.42
120,gidam,0,IN,1639988778,25,18.9833,81.4000,26.47,1.31
188,kyabe,0,TD,1639988833,21,9.4515,18.9449,28.81,4.17
282,umuahia,67,NG,1639988913,30,5.5263,7.4896,26.55,5.41
330,kadhan,0,PK,1639988954,13,24.4810,68.9859,26.28,1.66
426,broken hill,0,AU,1639989035,18,-31.9500,141.4333,25.84,4.12
470,usakos,63,NaN,1639989068,14,-22.0000,15.6000,30.66,2.56
533,wawa,0,NG,1639989118,22,9.9022,4.4192,25.12,3.57


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
hotel_df = myFavoriteWeather
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,temp,Wind Speed
0,malakal,97,SS,1639988753,20,9.5334,31.6605,25.46,4.42
1,gidam,0,IN,1639988778,25,18.9833,81.4000,26.47,1.31
2,kyabe,0,TD,1639988833,21,9.4515,18.9449,28.81,4.17
3,umuahia,67,NG,1639988913,30,5.5263,7.4896,26.55,5.41
4,kadhan,0,PK,1639988954,13,24.4810,68.9859,26.28,1.66
5,broken hill,0,AU,1639989035,18,-31.9500,141.4333,25.84,4.12
6,usakos,63,NaN,1639989068,14,-22.0000,15.6000,30.66,2.56
7,wawa,0,NG,1639989118,22,9.9022,4.4192,25.12,3.57


In [47]:
hotel_df['Hotel Name'] = ''

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "rankby": "prominence",
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get variables type from df
    lat = row['Lat']
    lng = row['Lng']
    city = row['City']
    params["location"] = f'{lat},{lng}'
        
    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {city} hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except:
        print(f"Missing field/result for {city}... skipping.")
        
    print("------------")

<ipython-input-47-4849ca58bcef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


Missing field/result for malakal... skipping.
------------
Closest gidam hotel is Durga Hotel Main Road Geedam.
------------


/Users/sonambhandari/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Missing field/result for kyabe... skipping.
------------
Closest umuahia hotel is Rixos Continental Resort.
------------
Missing field/result for kadhan... skipping.
------------
Closest broken hill hotel is Silver Haven.
------------
Closest usakos hotel is Nubeb Self-Catering Accommodation.
------------
Missing field/result for wawa... skipping.
------------


In [48]:
hotel_df = hotel_df[hotel_df['Hotel Name'].notna()]
hotel_df = hotel_df.rename(columns={"Latitude": "Lat", "Longitude": "Lng"})
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,temp,Wind Speed,Hotel Name
0,malakal,97,SS,1639988753,20,9.5334,31.6605,25.46,4.42,
1,gidam,0,IN,1639988778,25,18.9833,81.4000,26.47,1.31,Durga Hotel Main Road Geedam
2,kyabe,0,TD,1639988833,21,9.4515,18.9449,28.81,4.17,
3,umuahia,67,NG,1639988913,30,5.5263,7.4896,26.55,5.41,Rixos Continental Resort
4,kadhan,0,PK,1639988954,13,24.4810,68.9859,26.28,1.66,
5,broken hill,0,AU,1639989035,18,-31.9500,141.4333,25.84,4.12,Silver Haven
6,usakos,63,NaN,1639989068,14,-22.0000,15.6000,30.66,2.56,Nubeb Self-Catering Accommodation
7,wawa,0,NG,1639989118,22,9.9022,4.4192,25.12,3.57,


In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [51]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))